In [1]:
![ ! -f requirements.txt ] && git clone https://github.com/anudaweerasinghe/small-coder.git && cd small-coder

Cloning into 'small-coder'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 1), reused 17 (delta 1), pack-reused 0
Receiving objects: 100% (19/19), 28.42 KiB | 5.68 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
%%capture

! pip3 install -r small-coder/requirements.txt

In [3]:
# setup human eval and mbpp dataset
from datasets import load_dataset
humaneval = load_dataset("openai_humaneval", split="test")
mbpp = load_dataset("mbpp", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [16]:
def mbppToFnHeaderAndDocString(mbpp_datum):
  ref_implementation = mbpp_datum['code']
  prompt = mbpp_datum['text']

  fn_header = ref_implementation.split('\n', 1)[0]

  return f'{fn_header}\n    """\n    {prompt}\n    """'


In [64]:
def getHumanEvalTestsList(humaneval_datum):
  check_fn = humaneval_datum['test']

  lines = [line.strip() for line in check_fn.split('\n')]

  tests = list(filter(lambda string: string.startswith('assert'), lines))

  return tests

In [65]:
# View one example of humaneval and mbpp
print('=================HumanEval=================')
print(humaneval[0]['prompt'])
print(getHumanEvalTestsList(humaneval[0]))

print('===================MBPP===================')
print(mbppToFnHeaderAndDocString(mbpp[0]))
print(mbpp[0]['test_list'])

=================HumanEval=================
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False']
===================MBPP===================
def remove_Occ(s,ch): 
    """
    Write a python function to remove first and last occurrence of a given character from th

In [18]:
#initialize deepsek-coder-1.3b
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [42]:
# helper to generate code for agiven docstring
def generateCode(docstring):
  message = [
    {'role': 'user', 'content': 'Complete the following code. Do not generate test cases.'},
    { 'role': 'user', 'content': docstring  }
  ]

  inputs = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt").to(model.device)
  # tokenizer.eos_token_id is the id of <|EOT|> token
  outputs = model.generate(inputs, max_new_tokens=128000, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

  return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [43]:
humanevalExampleResult = generateCode(humaneval[0]['prompt'])
mbppExampleResult = generateCode(mbppToFnHeaderAndDocString(mbpp[0]))

print('=================HumanEval=================')
print(humanevalExampleResult)

print('===================MBPP===================')
print(mbppExampleResult)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


=================HumanEval=================
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

===================MBPP===================
def remove_Occ(s,ch): 
    """
    Write a python function to remove first and last occurrence of a given character from the string.
    """
    if ch in s:
        s = s.replace(ch, '', 1)
        s = s.replace(ch, '', 1)
    return s

